In [1]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
import numpy as np
import nltk
import collections
import re
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, layers, losses
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import f1_score

## Create pairs definition

In [5]:
def create_pairs(x, y):

  pairs_list = []
  labels_list = []

  for i in range(len(x)):
    for j in range(len(x)):
      if i == j:
        continue;
      if j < i:
        continue;
      pairs_list.append((x[i], x[j]))
      labels_list.append((y[i][0], y[j][0]))

  return np.array(pairs_list), np.array(labels_list)

## BERT Import


In [6]:
!pip install spacy-sentence-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.5 MB/s eta 0:00:00
  Created wheel for spacy-sentence-bert: filename=spacy_sentence_bert-0.1.2-py3-none-any.whl size=27201 sha256=88dde9633548ed3568bb52dc2792a7915af65eac06deef11da9fcc195e75f614
  Stored in directory: /root/.cache/pip/wheels/e9/3d/c9/a2e13b1a9dcaacca18d82bd7154ab95d1cea35d707c472a133
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=125e844df4b22482007b3

In [7]:
import spacy_sentence_bert
import spacy

In [8]:
nlp = spacy_sentence_bert.load_model('en_stsb_roberta_large')

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

## Contrastive loss definition - contrastive + softmax


In [23]:
# https://keras.io/examples/vision/siamese_contrastive/
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    x = tf.math.l2_normalize(x)
    y = tf.math.l2_normalize(y)
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    output = tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))
    return output

def cosine_sim(vects):
    x, y = vects
    x = tf.math.l2_normalize(x)
    y = tf.math.l2_normalize(y)

    return tf.multiply(x,y)

def loss(theta = 0.05, margin=0.5):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'constrastive_loss' function with data ('margin') attached.
    """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):

        """Calculates the constrastive loss.

        Arguments:
            y_true: List of labels, each label is of type float32.
            y_pred: List of predictions of same length as of y_true,
                    each label is of type float32.

        Returns:
            A tensor containing constrastive loss as floating point value.
        """

        # if (np.sum(y_true.numpy()) == len(y_true.numpy())) | (np.sum(y_true.numpy()) == 0):
        #   print("batch with only positives or negatives!!")
        #   square_pred = tf.math.square(y_pred)
        #   margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        #   return tf.math.reduce_mean(
        #       (1 - y_true) * square_pred + (y_true) * margin_square
        #   )
        # else:
        #   exp_vec = tf.math.exp(y_pred/theta)
        #   loss = tf.constant(-tf.math.log(tf.math.reduce_sum(exp_vec*y_true)/tf.math.reduce_sum(exp_vec*(1-y_true))))
        #   return loss

        # Previous contrastive loss
        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        loss = tf.math.reduce_mean(
              (1 - y_true) * square_pred + (y_true) * margin_square
        )
        return loss


    return contrastive_loss


## Analyze Reviews structure

In [10]:
np.random.seed(123)

In [11]:
df = pd.read_csv("/content/drive/MyDrive/NLP_Projekt/IMDB Dataset.csv").iloc[:100,:]
y = np.asarray(df.iloc[:,1])

In [12]:
df.review.iloc[3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [13]:
import re

review_y_total_list = []
review_total_list = []

for i in range(df.shape[0]):
  review = df.review.iloc[i].replace("<br /><br />", "")
  review_y = y[i]
  regex = re.compile('[,\<>/\|]')
  review = regex.sub('', review)
  review = review.replace("?",".")
  review_list = review.split(".")
  review_list = list(filter(None, review_list))
  review_y_list = [review_y for sentence in review_list]

  review_total_list.append(review_list)
  review_y_total_list.append(review_y_list)

In [14]:
review_y_total_list = [item for review_y_list in review_y_total_list for item in review_y_list]
review_total_list = [item for review_list in review_total_list for item in review_list]

In [15]:
len(review_total_list)

1187

In [16]:
# Drop reviews(sentences) with less than 50 letters
for i in range(len(review_total_list)):
  if len(review_total_list[i]) < 50:
    review_total_list[i] = None
    review_y_total_list[i] = None

review_total_list = [review for review in review_total_list if review is not None]
review_y_total_list = [y for y in review_y_total_list if y is not None]

len(review_total_list)

928

In [17]:
df_new = pd.DataFrame()
df_new["X"] = review_total_list
df_new["y"] = review_y_total_list
df_new

,X,y
0,One of the other reviewers has mentioned that ...,positive
1,They are right as this is exactly what happen...,positive
2,The first thing that struck me about Oz was it...,positive
3,Trust me this is not a show for the faint hea...,positive
4,This show pulls no punches with regards to dr...,positive
...,...,...
923,Then there is the graphics these are by far t...,positive
924,My only minor gripes is that the going upside ...,positive
925,The worst part is that you lose all of your l...,positive
926,Still these minor problems aside it's a super...,positive


## Perform constrastive learning


In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import f1_score


y = np.asarray(df_new.iloc[:,1])
y[y=="positive"] = 1
y[y=="negative"] = 0
y = y.reshape(-1,1).astype('float32')
X = np.array([nlp(df_new.iloc[i,0]).vector for i in range(df_new.shape[0])])

skf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=123)
skf.get_n_splits(X, y)

accuracy_list = []
f1_score_list = []

for train_index, test_index in skf.split(X, y):

  # Select random subsamples
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  X_train = np.array(X_train)
  X_test = np.array(X_test)
  X_train.shape

  df2 = pd.read_csv("/content/drive/MyDrive/NLP_Projekt/IMDB Dataset.csv").iloc[200:500,:]
  y_test = np.asarray(df2.iloc[:,1])
  y_test[y_test=="positive"] = 1
  y_test[y_test=="negative"] = 0
  y_test = y_test.reshape(-1,1).astype('float32')
  X_test = np.array([nlp(df2.iloc[i,0]).vector for i in range(df2.shape[0])])
  X_test = np.array(X_test)



  # create pairs

  x_train_pairs, y_train_labels = create_pairs(X_train, y_train)
  # x_test_pairs, y_test_labels = create_pairs(X_test, y_test)
  # if labels are the same -> label 0
  # if labels are not the same -> label 1
  y_train_labels = np.where(y_train_labels[:,0] == y_train_labels[:,1], 0, 1)

  # https://keras.io/examples/vision/siamese_contrastive/
  # Provided two tensors t1 and t2
  # Euclidean distance = sqrt(sum(square(t1-t2)))

  len_bag = x_train_pairs.shape[2]

  input = layers.Input((len_bag,))
  x = tf.keras.layers.BatchNormalization()(input)
  x = layers.Dense(2048)(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(1024)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(512)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(256)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(128)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(32)(x)

  embedding_network = tf.keras.Model(input, x)


  input_1 = layers.Input((len_bag,))
  input_2 = layers.Input((len_bag,))

  # As mentioned above, Siamese Network share weights between
  # tower networks (sister networks). To allow this, we will use
  # same embedding network for both tower networks.
  tower_1 = embedding_network(input_1)
  tower_2 = embedding_network(input_2)

  merge_layer = layers.Lambda(cosine_sim)([tower_1, tower_2])
  normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
  output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
  siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

  siamese.compile(loss=loss(), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=["accuracy"], run_eagerly=True)
  siamese.summary()

  x_train_1 = x_train_pairs[:,0,:]
  x_train_2 = x_train_pairs[:,1,:]

  # x_test_1 = x_test_pairs[:,0,:]
  # x_test_2 = x_test_pairs[:,1,:]

  x_train_1 = tf.convert_to_tensor(x_train_1)
  x_train_2 = tf.convert_to_tensor(x_train_2)

  # x_test_1 = tf.convert_to_tensor(x_test_1)
  # x_test_2 = tf.convert_to_tensor(x_test_2)

  y_train_labels = tf.convert_to_tensor(y_train_labels)
  # y_test_labels = tf.convert_to_tensor(y_test_labels)


  y_train_labels = tf.cast(y_train_labels, tf.float32)
  # y_test_labels = tf.cast(y_test_labels, tf.float32)

  # x_test_1 = tf.cast(x_test_1, tf.float32)
  # x_test_2 = tf.cast(x_test_2, tf.float32)

  x_train_1 = tf.cast(x_train_1, tf.float32)
  x_train_2 = tf.cast(x_train_2, tf.float32)


  history = siamese.fit(
      [x_train_1, x_train_2],
      y_train_labels,
      batch_size=32,
      epochs=3
  )

  # a = siamese.predict([x_test_1, x_test_2])
  # results = pd.DataFrame(a)
  # results["true"] = y_test_labels[:,1]
  # results.iloc[:50,:]

  # plt.figure(figsize=(8,10))
  # plt.hist(results[results["true"] == 0][0], color="orange", alpha = 0.5)
  # plt.hist(results[results["true"] == 1][0], color="skyblue", alpha = 0.5)
  # plt.legend(["class 0", "class 1"])
  # plt.title("Probability vs actual")
  # plt.show()

  preds = []

  for i in range(len(X_test)):
      pairs_list = []

      for j in range(len(X_train)):
          pairs_list.append((X_test[i], X_train[j]))

      pairs_list = np.array(pairs_list)

      b = siamese.predict([pairs_list[:,0,:], pairs_list[:,1,:]])
      preds.append(int(b[np.where(y_train.flatten() == 0)].mean() > b[np.where(y_train.flatten() == 1)].mean()))

  accuracy_list.append(np.mean(y_test.flatten()==preds))
  print("Accuracy:")
  print(np.mean(y_test.flatten()==preds))
  f1_score_list.append(f1_score(y_test.flatten(), preds))


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 model (Functional)             (None, 32)           4902304     ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 32)           0           ['model[0][0]',            

KeyboardInterrupt: ignored

In [ ]:
accuracy_list

[0.65]

In [ ]:
f1_score_list

[0.6037735849056602]

## Nested voting approach 

In [20]:
y = np.asarray(df_new.iloc[:,1])
y[y=="positive"] = 1
y[y=="negative"] = 0
y = y.reshape(-1,1).astype('float32')
X = np.array([nlp(df_new.iloc[i,0]).vector for i in range(df_new.shape[0])])

skf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=123)
skf.get_n_splits(X, y)

accuracy_list = []
f1_score_list = []

df2 = pd.read_csv("/content/drive/MyDrive/NLP_Projekt/IMDB Dataset.csv").iloc[200:500,:]

# create sentence obs in test set
review_y_total_list_test = []
review_total_list_test = []
sentence_review_indices = []

for i in range(df2.shape[0]):
  review = df2.review.iloc[i].replace("<br /><br />", "")
  review_y = df2.iloc[i,1]
  regex = re.compile('[,\<>/\|]')
  review = regex.sub('', review)
  review = review.replace("?",".")
  review_list = review.split(".")
  review_list = list(filter(None, review_list))
  review_list_filtered = [sentence for sentence in review_list if len(sentence) >= 50]
  sentence_review_index = [i for sentence in review_list_filtered]
  review_y_list = [review_y for sentence in review_list_filtered]

  review_total_list_test.append(review_list_filtered)
  review_y_total_list_test.append(review_y_list)
  sentence_review_indices.append(sentence_review_index)


review_y_total_list_test = [item for review_y_list in review_y_total_list_test for item in review_y_list]
review_total_list_test = [item for review_list in review_total_list_test for item in review_list]
sentence_review_indices = [item for sentence_index in sentence_review_indices for item in sentence_index]

df3 = pd.DataFrame()
df3["X"] = review_total_list_test 
df3["y"] = review_y_total_list_test
df3["review_index"] = sentence_review_indices
df3

y_test = np.asarray(df3.iloc[:,1])
y_test[y_test=="positive"] = 1
y_test[y_test=="negative"] = 0
y_test = y_test.reshape(-1,1).astype('float32')
X_test = np.array([nlp(df3.iloc[i,0]).vector for i in range(df3.shape[0])])
X_test = np.array(X_test)

In [21]:
df3

,X,y,review_index
0,Interesting and short television movie describ...,0,0
1,Film is currently very topical given the publ...,0,0
2,The film does a good job of sparking viewers' ...,0,0
3,The portrayal of Ovitz was particularly compe...,0,0
4,Still many of the characters were only very br...,0,0
...,...,...,...
2996,However it's easy to ignore this fault and it...,1,299
2997,Although A Bug's Life may not reach the dizzyi...,1,299
2998,Pixar have proved that they're not just a one...,1,299
2999,They raise the bar endlessly and when someone...,1,299


In [28]:
pred_total = []

for train_index, test_index in skf.split(X, y):

  # Select random subsamples
  X_train, X_test_dummy = X[train_index], X[test_index]
  y_train, y_test_dummy = y[train_index], y[test_index]
  X_train = np.array(X_train)
  X_test_dummy = np.array(X_test_dummy)
  X_train.shape



  # create pairs

  x_train_pairs, y_train_labels = create_pairs(X_train, y_train)
  # x_test_pairs, y_test_labels = create_pairs(X_test, y_test)
  # if labels are the same -> label 0
  # if labels are not the same -> label 1
  y_train_labels = np.where(y_train_labels[:,0] == y_train_labels[:,1], 0, 1)

  # https://keras.io/examples/vision/siamese_contrastive/
  # Provided two tensors t1 and t2
  # Euclidean distance = sqrt(sum(square(t1-t2)))

  len_bag = x_train_pairs.shape[2]

  input = layers.Input((len_bag,))
  x = tf.keras.layers.BatchNormalization()(input)
  x = layers.Dense(2048)(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(1024)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(512)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(256)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(128)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(32)(x)

  embedding_network = tf.keras.Model(input, x)


  input_1 = layers.Input((len_bag,))
  input_2 = layers.Input((len_bag,))

  # As mentioned above, Siamese Network share weights between
  # tower networks (sister networks). To allow this, we will use
  # same embedding network for both tower networks.
  tower_1 = embedding_network(input_1)
  tower_2 = embedding_network(input_2)

  merge_layer = layers.Lambda(cosine_sim)([tower_1, tower_2])
  normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
  output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
  siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

  siamese.compile(loss=loss(), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=["accuracy"], run_eagerly=True)
  siamese.summary()

  x_train_1 = x_train_pairs[:,0,:]
  x_train_2 = x_train_pairs[:,1,:]

  # x_test_1 = x_test_pairs[:,0,:]
  # x_test_2 = x_test_pairs[:,1,:]

  x_train_1 = tf.convert_to_tensor(x_train_1)
  x_train_2 = tf.convert_to_tensor(x_train_2)

  # x_test_1 = tf.convert_to_tensor(x_test_1)
  # x_test_2 = tf.convert_to_tensor(x_test_2)

  y_train_labels = tf.convert_to_tensor(y_train_labels)
  # y_test_labels = tf.convert_to_tensor(y_test_labels)


  y_train_labels = tf.cast(y_train_labels, tf.float32)
  # y_test_labels = tf.cast(y_test_labels, tf.float32)

  # x_test_1 = tf.cast(x_test_1, tf.float32)
  # x_test_2 = tf.cast(x_test_2, tf.float32)

  x_train_1 = tf.cast(x_train_1, tf.float32)
  x_train_2 = tf.cast(x_train_2, tf.float32)


  history = siamese.fit(
      [x_train_1, x_train_2],
      y_train_labels,
      batch_size=32,
      epochs=2
  )

  # a = siamese.predict([x_test_1, x_test_2])
  # results = pd.DataFrame(a)
  # results["true"] = y_test_labels[:,1]
  # results.iloc[:50,:]

  # plt.figure(figsize=(8,10))
  # plt.hist(results[results["true"] == 0][0], color="orange", alpha = 0.5)
  # plt.hist(results[results["true"] == 1][0], color="skyblue", alpha = 0.5)
  # plt.legend(["class 0", "class 1"])
  # plt.title("Probability vs actual")
  # plt.show()

  preds = []

  for i in range(len(X_test)):
      pairs_list = []

      for j in range(len(X_train)):
          pairs_list.append((X_test[i], X_train[j]))

      pairs_list = np.array(pairs_list)

      b = siamese.predict([pairs_list[:,0,:], pairs_list[:,1,:]])
      preds.append(int(b[np.where(y_train.flatten() == 0)].mean() > b[np.where(y_train.flatten() == 1)].mean()))

  accuracy_list.append(np.mean(y_test.flatten()==preds))
  print("Accuracy:")
  print(np.mean(y_test.flatten()==preds))
  f1_score_list.append(f1_score(y_test.flatten(), preds))

  pred_total.append(preds)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
15/15 [==============================] - 0s 22ms/step
Accuracy:
0.6137954015328224
Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_32 (InputLayer)          [(None, 1024)]       0           []                               
                                                                                                  
 input_33 (InputLayer)          [(None, 1024)]       0           []                               
                                                                                                  
 model_20 (Functional)          (None, 32)           4902304     ['input_32[0][0]',               
                                                                  'input_33[0][0]']               
                                          

In [29]:
pred_total_flat = [item for pred in pred_total for item in pred]

def calculate_final_values(pred_total, y_test, sentence_review_indices, i):

  df_results = pd.DataFrame()
  df_results["y_true"] = y_test.flatten()
  df_results["predicted"] = pred_total[i]
  df_results["review_indices"] = sentence_review_indices
  df_final = pd.DataFrame(df_results.groupby(["review_indices"])["predicted"].mean()).reset_index()
  df_final["y_true"] = df2.iloc[:,1].to_numpy()
  df_final["y_label"] = np.where(df_final["y_true"]=="positive",1,0 )
  df_final["y_pred"] = np.where(df_final["predicted"] < 0.5, 0, 1)


  return (df_final.y_label == df_final.y_pred).mean(), f1_score(df_final.y_label, df_final.y_pred)

In [30]:
final_acc = []
final_f1 = []

# for every CV run
for i in range(4):
  run_acc, f1_score_val = calculate_final_values(pred_total, y_test, sentence_review_indices, i)
  final_acc.append(run_acc)
  final_f1.append(f1_score_val)
print("ACC: ", final_acc)
print("F1", final_f1)

ACC:  [0.7066666666666667, 0.6833333333333333, 0.6933333333333334, 0.7466666666666667]
F1 [0.7006802721088436, 0.702194357366771, 0.684931506849315, 0.732394366197183]
